## Setup instructions
1. Setup your AWS environment (set `AWS_REGION`) as appropriate to generate a client in the right account/region
1. Set the environment variable `ENDPOINT_NAME` to point to your Navigator model, or update the cell below
1. Run the below to verify you can generate data with Navigator

In [17]:
%%capture
!pip install -Uqq boto3==1.35.20

In [ ]:
%env AWS_REGION=us-west-2
%env AWS_PROFILE=UPDATE_ME
%env ENDPOINT_NAME=UPDATE_ME

In [2]:
import json
import os

import boto3

boto3.setup_default_session(profile_name=os.environ.get("AWS_PROFILE"),
                            region_name=os.environ.get("AWS_REGION"))

model_name = os.environ.get("ENDPOINT_NAME")
client = boto3.client("sagemaker-runtime")
body = {
    "messages": [
        {
            "role": "user",
            "content": "Can you generate a dataset with first_name, last_name and favorite_color as columns.",
        }
    ],
    "n": 2,
    "model": "gretelai/auto",
    "gretel": {
        "ref_data": {
            "sample_data": {
                "table_headers": [
                    "first_name",
                    "last_name",
                    "favorite_color",
                ],
                "table_data": [
                    {
                        "first_name": "Lea",
                        "last_name": "Martin",
                        "favorite_color": "Red",
                    }
                ],
            }
        }
    },
}

In [3]:
# Non streaming case
body["stream"] = False

response = client.invoke_endpoint(
    EndpointName=model_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(body),
)

response_json = json.load(response["Body"])

print(json.dumps(response_json, indent=2))

assert "choices" in response_json
assert "role" in response_json["choices"][0]["message"]
assert "content" in response_json["choices"][0]["message"]

{
  "id": "48b0e1bb-b92c-4d14-aa88-cacc11b04911",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\"table_headers\":[\"first_name\",\"last_name\",\"favorite_color\"],\"table_data\":[{\"first_name\":\"Elena\",\"last_name\":\"Garcia\",\"favorite_color\":\"Turquoise\"}]}\n{\"table_headers\":[\"first_name\",\"last_name\",\"favorite_color\"],\"table_data\":[{\"first_name\":\"Kaito\",\"last_name\":\"Liang\",\"favorite_color\":\"Indigo\"}]}\n",
        "refusal": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1730924031,
  "model": "gretelai/auto",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 74,
    "prompt_tokens": 21,
    "total_tokens": 95,
    "completion_tokens_details": null
  },
  "gretel": {
    "provenance": []
  }
}


In [4]:
# Streaming case
body["stream"] = True

stream_response = client.invoke_endpoint_with_response_stream(
    EndpointName=model_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(body),
)

last_chunk = ""
for chunk in stream_response["Body"]:
    proc_chunk = (
        chunk["PayloadPart"]["Bytes"].decode("utf_8").replace("data:", "").strip()
    )
    try:
        payload = last_chunk + proc_chunk
        response_json = json.loads(payload)
        last_chunk = ""
        print(response_json)
        assert "choices" in response_json
        assert "delta" in response_json["choices"][0]
        assert "index" in response_json["choices"][0]
        assert "role" in response_json["choices"][0]["delta"]
        assert "content" in response_json["choices"][0]["delta"]
    except json.decoder.JSONDecodeError:
        last_chunk = proc_chunk

{'id': 'e2ef0213-a295-42f2-b8b0-faa614db8774', 'choices': [{'delta': {'content': '{"table_headers":["first_name","last_name","favorite_color"],"table_data":[{"first_name":"Ahmed","last_name":"Patel","favorite_color":"Blue"}]}\n', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730924037, 'model': 'gretelai/auto', 'object': 'chat.completion.chunk', 'service_tier': None, 'system_fingerprint': None, 'usage': None, 'gretel': None}
{'id': 'e2ef0213-a295-42f2-b8b0-faa614db8774', 'choices': [{'delta': {'content': '{"table_headers":["first_name","last_name","favorite_color"],"table_data":[{"first_name":"Léa","last_name":"Santos","favorite_color":"Green"}]}\n', 'function_call': None, 'refusal': None, 'role': 'assistant', 'tool_calls': None}, 'finish_reason': None, 'index': 0, 'logprobs': None}], 'created': 1730924037, 'model': 'gretelai/auto', 'object': 'chat.completion.chunk', 'service_tier': N